In [ ]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4
import tqdm
import sys
import inspect
# chromeのwebdriver自動更新用
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [ ]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [ ]:
# 1
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_scraping_companies = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year, "出力中")
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(dfs_list)):
            dfs_list[i]["上場年"] = f"{year}"
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list[i]])
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year, "出力中")
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        # サイト更新時にページのtableのデザインが変更していないかチェック
        if year >= 2022:
            try:
                if dfs_list[0].columns[0] != '企業名':
                    raise ValueError("銘柄一覧から取得したtableの値が「企業名」ではありません")
                elif dfs_list[1].columns[0] != '総合評価':
                    raise ValueError("銘柄一覧から取得したtableの値が「総合評価」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")   
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(dfs_list) , 2):
            dfs_list_con = pd.concat([dfs_list[i], dfs_list[i+1]], axis=1)
            dfs_list_con["上場年"] = f"{year}"
            if year == 2019:
                dfs_list_con = dfs_list_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                dfs_list_con = dfs_list_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list_con],ignore_index=True)
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)

In [ ]:
df_scraping_companies

In [ ]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_scraping_url = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_scraping_url = df_scraping_url.rename(columns={0:"url"})
# 間違えているurlを変更
df_scraping_url.url[df_scraping_url.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"
df_scraping_url = df_scraping_url.rename_axis('index').reset_index()
df_scraping_url = df_scraping_url.rename(columns={'index':'company_name'})

In [ ]:
df_scraping_url

In [ ]:
# 2
# seleniumでのデータ取得用ループ　評価
# chromeのwebdriverのinstallしブラウザを起動する 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
def make_company_info(url):
    val_list = []
    driver.get(url)
    #seleniumの場合ここでsleepしないと正しく取得出来ない場合がある
    time.sleep(np.random.randint(100,120)/100)
    # HTMLを文字コードをUTF-8に変換してから取得します。
    html_res = driver.page_source.encode('utf-8')
#ここでシステムエラー処理する？
    # if html_res.status_code != 200:
    #     print("requests.getでのurlのアクセスができていません")
    #     lineno = inspect.currentframe().f_lineno
    #     print(f"エラーが発生しました。行番号: {lineno}")
    #     sys.exit()
    soup = bs4(html_res, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    val_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list.extend([np.nan,np.nan,np.nan,np.nan])
        print(f"成長性listのエラー{code}")
    print(code)
    # アンケート調査結果の取得
    print(len(soup.find_all("div",class_="vote-bar")))
    # 6行の場合
    if len(soup.find_all("div",class_="vote-bar")) == 6:
        vote_list = []
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 5行の場合
    elif len(soup.find_all("div",class_="vote-bar")) == 5:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 10行の場合 5行のアンケートがなぜか２つ表示されている銘柄
    elif len(soup.find_all("div",class_="vote-bar")) == 10:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        number_of_votes_list = number_of_votes_list[0:5]
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # アンケートがない場合
    elif len(soup.find_all("div",class_="vote-bar")) == 0:
        val_list.extend([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    # その他の行数エラー検知
    else:
        print("票の価格帯の数が分岐にありません")
        driver.quit()
        raise Exception("Error: アンケートの価格帯の数がifの分岐にありません")
    # 今まで取得した情報をリストに格納
    evaluation_list.append(val_list)
df_scraping_evaluation = df_scraping_url.copy()
url_list = df_scraping_url.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:]] #ここで検証のurl数変更！
print(evaluation_list)
df_company_01 = pd.DataFrame(evaluation_list,columns=[
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
])
df_scraping_evaluation[df_company_01.columns] = df_company_01
print(df_scraping_evaluation)
driver.quit()

In [11]:
# 3
def make_company_info_2(url):
    dfs_table_list = pd.read_html(url)
    print(dfs_table_list[1].iloc[0,1])
    time.sleep(np.random.randint(100,120)/100)
    # サイト更新時にデザイン変更がないか検知する（現状ワシントンホテルのみ違うためスルー）
    # 回避urlリスト
    avoidance_url_list = [
        "https://www.ipokiso.com//company/2019/washingtonhotel.html",#[0]に１つ余分に優待情報のtableある
        "https://www.ipokiso.com//company/2020/gmo-fg.html",#table[6][7]が[7][8]にずれている
        "https://www.ipokiso.com//company/2021/frontier.html",#table[6]が無い
        "https://www.ipokiso.com//company/2021/geolocation.html",#table[6]が無い
        ]
    if url not in avoidance_url_list:
        try:
            if dfs_table_list[1].iloc[0,0] != "会社名":
                raise ValueError("銘柄一覧から取得したtableの値が「会社名」ではありません")
            elif dfs_table_list[2].iloc[0,0] != '想定価格':
                raise ValueError("銘柄一覧から取得したtableの値が「想定価格」ではありません")               
            elif dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                raise ValueError("銘柄一覧から取得したtableの値が「抽選申込期間」ではありません")
            elif not '公募' in dfs_table_list[4].iloc[0,0]:
                raise ValueError("銘柄一覧から取得したtableの値が「公募株数等」ではありません")
            elif dfs_table_list[5].iloc[0,1] != '証券会社名':
                raise ValueError("銘柄一覧から取得したtableの値が「証券会社名」ではありません")
        except ValueError as e:
            # エラーが発生した場合の処理
            print("Error: {}".format(e))
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生した行番号: {lineno}")
            print(f"エラーが発生したURL:{url}")
            print(dfs_table_list)
            sys.exit()
        # 会社名にREITと投資法人という文字列が無いことを判定する
        if not ("REIT" in dfs_table_list[1].iloc[0,1]) and not ("投資法人" in dfs_table_list[1].iloc[0,1]):
            try:
                if dfs_table_list[6].columns[0] != '株主名':
                    raise ValueError("銘柄一覧から取得したtableの値が「株主名」ではありません")
                elif not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    raise ValueError("銘柄一覧から取得したtableの値が「財務データ等」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")
                print(f"エラーが発生したURL:{url}")
                print(dfs_table_list)
                sys.exit()

    # ここからdfsの結合、整形コード
    df_con_list = []
    for i in range(len(dfs_table_list)):
        # 基本情報
        if i == 1:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if df_table.iloc[0,0] != "会社名":
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：基本情報
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_1)
            # 表記の揺れ統一のための修正

        # IPO日程と価格決定（初値予想）
        if i == 2:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if dfs_table_list[2].iloc[0,0] != '想定価格':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：IPO日程と価格決定（初値予想）
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_2)
            # 表記の揺れ統一のための修正
            
        # IPOスケジュール
        if i == 3:
            # tableの位置指定 # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                df_table = dfs_table_list[i + 1]
                print(url)
            
            # df整形：IPOスケジュール
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_3 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_3)
            # 表記の揺れ統一のための修正
        # IPO当選株数
        if i == 4:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if not '公募' in dfs_table_list[4].iloc[0,0]:
                df_table = dfs_table_list[i + 1]
                print(url)
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = dfs_table_list[i + 1]
            
            # df整形：IPO当選株数
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_4 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_4)
            # 表記の揺れ統一のための修正

        # 幹事証券リスト（管理人独自予想あり）
        if i == 5:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[5].iloc[0,1] != '証券会社名':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：幹事証券リスト（管理人独自予想あり）
            df_table_5 = pd.DataFrame()
            for j in range(1,len(df_table.iloc[:,0])):
                val = df_table.iloc[j,:].values
                cols = df_table.iloc[0,:]
                cols[0] = "幹事種類"
                cols = [f"{col}_{j}" for col in cols]
                df_add = pd.DataFrame([val],columns=cols)
                df_table_5 = pd.concat([df_table_5, df_add], axis=1)
            df_con_list.append(df_table_5)

        # 株主構成、ロックアップなど
        if i == 6:
            if dfs_table_list[1].iloc[0,0] == "会社名":
                company_name = dfs_table_list[1].iloc[0,1]
            elif dfs_table_list[2].iloc[0,0] == "会社名":
                company_name = dfs_table_list[2].iloc[0,1]
            if not ("REIT" in company_name or "投資法人" in company_name or  ("福証" in company_name and not "東証" in company_name)):
                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if dfs_table_list[6].columns[0] != '株主名':
                    df_table = dfs_table_list[i + 1]
                    print(url)
                df_table_6 = pd.DataFrame()
                for k in range(0,len(df_table.iloc[:,0])):
                    val = df_table.iloc[k,:].values
                    cols = df_table.columns
                    cols = [f"{col}_{k+1}" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_6 = pd.concat([df_table_6, df], axis=1)
                df_con_list.append(df_table_6)

            # 企業業績のデータ（5年分）
        if i == 7:
            if not ("REIT" in company_name or "投資法人" in company_name):

                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    df_table = dfs_table_list[i + 1]
                    print(url)
                
                df_table_7 = pd.DataFrame()
                for h in range(1,len(df_table.T.iloc[:,0])):
                    val = df_table.T.iloc[h,:].values
                    # カラム名の誤字修正
                    cols = df_table.T.iloc[0,:]
                    cols[0] = "年月"
                    cols = [f"{col}_{h}年目" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_7 = pd.concat([df_table_7, df], axis=1)
                df_con_list.append(df_table_7)


            # 全件のdfに追加していく
    df_table_all = pd.concat(df_con_list,axis=1)
    print(url)
    print("finish")
    return df_table_all
df_scraping_table_all = pd.DataFrame()
url_list = df_scraping_url.url.values
for url in url_list[0:]:#ここでurl数変更
    df_table_all = make_company_info_2(url) 
    df_scraping_table_all = pd.concat([df_scraping_table_all,df_table_all],axis=0,ignore_index=True)
print(df_scraping_table_all)

https://www.ipokiso.com//company/2021/arrplanner.html
finish
サイバートラスト（4498）　【東証マザーズ】
https://www.ipokiso.com//company/2021/cybertrust.html
finish
ステムセル研究所（7096）　【東証マザーズ】
https://www.ipokiso.com//company/2021/stemcell.html
finish
アルマード（4932）　【JASDAQスタンダード】
https://www.ipokiso.com//company/2021/almado.html
finish
松屋アールアンドディ（7317）　【東証マザーズ】
https://www.ipokiso.com//company/2020/matsuyard.html
finish
Macbee Planet[マクビープラネット]（7095）　【東証マザーズ】
https://www.ipokiso.com//company/2020/macbee-planet.html
finish
NexTone[ネクストーン]（7094）　【東証マザーズ】
https://www.ipokiso.com//company/2020/nex-tone.html
finish
ニッソウ（1444）　【名証セントレックス】
https://www.ipokiso.com//company/2020/nisso.html
finish
サイバーセキュリティクラウド（4493）　【東証マザーズ】
https://www.ipokiso.com//company/2020/cscloud.html
finish
アディッシュ（7093）　【東証マザーズ】
https://www.ipokiso.com//company/2020/adish.html
finish
ヴィス（5071）　【東証マザーズ】
https://www.ipokiso.com//company/2020/vis.html
finish
ペルセウスプロテオミクス（4882）　【東証マザーズ】
https://www.ipokiso.com//company/2021/ppmx.html
finish
リバーホール

In [12]:
# 時短のためcode含めたcsv読み込み　コード完成後に削除する
# df_scraping_evaluation = pd.read_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_url.csv", index_col=0)
# df_scraping_evaluation

12/15 dfの作成　4つ

In [ ]:
df_scraping_companies

,上場日,企業名,銘柄 コード,上場市場,申し込み 期間,当たり本数,想定価格,仮条件,公募価格,初値,初値 上昇率,狙い目証券,上場年,総合 評価,総合評価
0,12/22,アイセイ薬局,3170,JASDAQ スタンダード,12/6 ～12/12,"2,490本","3,460円","3,300円 ～3,460円","3,460円","3,460円",±0円 （0.0％）,SMBC日興 SBI カブドットコム 楽天,2011,NaN,NaN
1,12/22,ミサワ,3169,東証 マザーズ,12/7 ～12/13,"6,000本",860円,760円 ～860円,860円,"1,209円",+349円 （40.6％）,SMBC日興 （副） SBI 松井 カブドットコム,2011,NaN,NaN
2,12/21,スターフライヤー,9206,東証 2部,12/5 ～12/9,"2,000本","3,180円","2,900円 ～3,180円","3,180円","5,500円","+2,320円 （73.0％）",SMBC日興 （副） エイチエス マネックス カブドットコム 東海東京,2011,NaN,NaN
3,12/20,新田ゼラチン,4977,東証,12/5 ～12/9,"36,280本",570円,500円 ～570円,500円,462円,-38円 （-7.6％）,SMBC日興（主） SBI,2011,NaN,NaN
4,12/20,カイオム・ バイオサイエンス,4583,東証 マザーズ,12/2 ～12/8,"7,770本","2,700円","2,100円 ～2,700円","2,700円","2,480円",-220円 （-8.1％）,SBI 岩井コスモ むさし カブドットコム 楽天,2011,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,2/17,エッジテクノロジー (4268),NaN,東証マザーズ,2/1～2/7,"34,759本（多い）",690円（低価格）,300円～350円,350円,694円,+344円（+98.3％）,SMBC日興（主） 野村（副） SBI 楽天 松井 DMM株,2022,NaN,NaN
1051,2/9,ライトワークス (4267),NaN,東証マザーズ,1/21～1/27,"3,680本","2,100円","1,900円～2,100円","2,100円","3,000円",+900円（+42.9％）,SMBC日興（副） SBI 楽天 マネックス 岡三オンライン DMM株,2022,NaN,NaN
1052,2/4,ビッグツリーテクノロジー＆コンサルティング (4266) 上場中止,NaN,東証マザーズ,1/20～1/26,"42,734本（多い）","5,000円",-,-,-,-,大和（主） SMBC日興（副） 野村 SBI マネックス 岩井 楽天 松井 CONNECT ...,2022,NaN,NaN
1053,2/4,セイファート (9213),NaN,JASDAQスタンダード,1/19～1/25,"8,574本","1,120円","1,060円～1,120円","1,120円","1,030円",-90円（-8.0％）,大和（副） 岩井 マネックス CONNECT DMM株,2022,NaN,NaN


In [13]:
df_scraping_url

,company_name,url
0,アイセイ薬局,https://www.ipokiso.com//company/2011/aisei.html
1,ミサワ,https://www.ipokiso.com//company/2011/unico.html
2,スターフライヤー,https://www.ipokiso.com//company/2011/starflye...
3,新田ゼラチン,https://www.ipokiso.com//company/2011/nitta-ge...
4,カイオム・\r\nバイオサイエンス,https://www.ipokiso.com//company/2011/chiome.html
...,...,...
1009,エッジテクノロジー,https://www.ipokiso.com//company/2022/edge-tec...
1010,ライトワークス,https://www.ipokiso.com//company/2022/lightwor...
1011,ビッグツリーテクノロジー＆コンサルティング,https://www.ipokiso.com//company/2022/bigtreet...
1012,セイファート,https://www.ipokiso.com//company/2022/seyfert....


In [25]:
df_scraping_evaluation

,company_name,url,code,成長性,割安性,話題性,総合評価,初値予想_+200%以上,初値予想_+100%以上+200%未満,初値予想_+50%以上+100%未満,初値予想_+20%以上+50%未満,初値予想_+0%以上+20%未満,初値予想_+0%未満
0,アイセイ薬局,https://www.ipokiso.com//company/2011/aisei.html,3170,◎,〇,〇,B,NaN,NaN,NaN,NaN,NaN,NaN
1,ミサワ,https://www.ipokiso.com//company/2011/unico.html,3169,〇,△,〇,C,NaN,NaN,NaN,NaN,NaN,NaN
2,スターフライヤー,https://www.ipokiso.com//company/2011/starflye...,9206,〇,◎,〇,B,NaN,NaN,NaN,NaN,NaN,NaN
3,新田ゼラチン,https://www.ipokiso.com//company/2011/nitta-ge...,4977,△,〇,△,D,NaN,NaN,NaN,NaN,NaN,NaN
4,カイオム・\r\nバイオサイエンス,https://www.ipokiso.com//company/2011/chiome.html,4583,△,△,△,D,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,エッジテクノロジー,https://www.ipokiso.com//company/2022/edge-tec...,4268,〇,△,〇,D,92.0,46.0,187.0,102.0,53.0,244.0
1010,ライトワークス,https://www.ipokiso.com//company/2022/lightwor...,4267,◎,△,〇,C,116.0,28.0,53.0,59.0,103.0,224.0
1011,ビッグツリーテクノロジー＆コンサルティング,https://www.ipokiso.com//company/2022/bigtreet...,4266,〇,△,〇,C,69.0,16.0,15.0,28.0,79.0,255.0
1012,セイファート,https://www.ipokiso.com//company/2022/seyfert....,9213,△,△,〇,D,53.0,16.0,19.0,40.0,80.0,192.0


In [27]:
df_scraping_table_all

,会社名,会社URL,狙い目証券会社,想定価格,仮条件,公募価格,初値予想（独自）,初値,抽選申込期間,当選発表日,...,税引前利益（千米ドル）_5年目,当期利益（千米ドル）_5年目,純資産額 （千米ドル）_5年目,1株あたりの純資産額（米ドル）_5年目,1株あたりの純利益（米ドル）_5年目,1株あたりの当期利益（米ドル）_1年目,1株あたりの当期利益（米ドル）_2年目,1株あたりの当期利益（米ドル）_3年目,1株あたりの当期利益（米ドル）_4年目,1株あたりの当期利益（米ドル）_5年目
0,アイセイ薬局（3170） 【JASDAQスタンダード】,http://www.aisei.co.jp/,SMBC日興証券（副幹事）、SBI証券,"3,460円","3,300円 ～ 3,460円","3,460円",-,"3,460円（公募価格比 ±0％）",12月6日（火）～12月12日（月）,12月13日（火）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ミサワ（3169） 【JASDAQスタンダード】,http://www.unico-fan.co.jp/,SMBC日興証券（副幹事）、SBI証券、松井証券,860円,760円 ～ 860円,860円,-,"1,209円（公募価格比 +40.6％）",12月7日（水）～12月13日（火）,12月14日（水）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,スターフライヤー（9206） 【東証2部】,http://www.starflyer.jp/,SMBC日興証券（副幹事）、マネックス証券 カブドットコム証券,"3,180円","2,900円 ～ 3,180円","3,180円",-,"5,500円（公募価格比 +73.0％）",12月5日（月）～12月9日（金）,12月12日（月）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,新田ゼラチン（4977） 【東証2部】,http://www.nitta-gelatin.co.jp/,SMBC日興証券（主幹事）、SBI証券,570円,500円 ～ 570円,500円,-,462円（公募価格比 -7.6％）,12月5日（月）～12月9日（金）,12月12日（月）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,カイオム・バイオサイエンス（4583） 【東証マザーズ】,http://www.chiome.co.jp/,SBI証券、むさし証券、カブドットコム証券,"2,700円","2,100円 ～ 2,700円","2,700円",-,"2,480円（公募価格比 -8.1％）",12月2日（金）～12月8日（木）,12月9日（金）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,エッジテクノロジー（4268） 【東証マザーズ】,https://www.edge-tech.co.jp/,SMBC日興証券（主幹事）、野村證券（副幹事）、SBI証券、楽天証券、松井証券、DMM株,690円,300円 ～ 350円,350円,322円 ～ 550円（2月8日時点）,694円 （公募価格比+344円 +98.3％）,2月1日(火)～2月7日(月),2月8日(火),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,ライトワークス（4267） 【東証マザーズ】,https://www.lightworks.co.jp/,SMBC日興証券（副幹事）、SBI証券、楽天証券、マネックス証券、岡三オンライン、DMM株,"2,100円","1,900円 ～ 2,100円","2,100円","2,100円 ～ 2,900円（1月19日時点）","3,000円 （公募価格比+900円 +42.9％）",1月21日(金)～1月27日(木),1月28日(金),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,ビッグツリーテクノロジー＆コンサルティング（4266） 【東証マザーズ】,https://www.bigtreetc.com/,大和証券（共同主幹事）、SMBC日興証券（副幹事）、野村證券、SBI証券、マネックス証券、岩...,"5,000円",未発表,未発表,"5,000円 ～ 6,000円（12月28日時点）",-,1月20日(木)～1月26日(水),1月27日(木),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,セイファート（9213） 【JASDAQスタンダード】,https://www.seyfert.co.jp/,大和証券（副幹事）、岩井コスモ証券、マネックス証券、CONNECT、DMM株,"1,120円","1,060円 ～ 1,120円","1,120円","1,030円 ～ 1,500円（1月26日時点）","1,030円 （公募価格比-90円 -8.0％）",1月19日(水)～1月25日(火),1月26日(水),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # 評価+ 会社設立年 + 市場(G,P,S,…)　+ IPO日程と価格決定(初値予測) + 初値予想アンケート
# # df_scraping_evaluation[
#     "code",
#     "成長性",
#     "割安性",
#     "話題性",
#     "総合評価",
#     "初値予想_+200%以上",
#     "初値予想_+100%以上+200%未満",
#     "初値予想_+50%以上+100%未満",
#     "初値予想_+20%以上+50%未満",
#     "初値予想_+0%以上+20%未満",
#     "初値予想_+0%未満",
# ]
# df_scraping_table_all["会社設立",
#     "会社名",#市場に変形する
#     "想定価格",
#     "仮条件",
#     "公募価格",
#     "初値予想（独自）",
#     "初値"
# ]


# # 評価 is これ　table[0(selenium),1,1,2,無(selenium)]

In [40]:
df_assessments = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満"
]],
df_scraping_table_all.loc[:, [
    "会社設立",
    "会社名",#市場に変形する
    "想定価格",
    "仮条件",
    "公募価格",
    "初値予想（独自）",
    "初値"
]]],axis=1)
df_assessments

,code,成長性,割安性,話題性,総合評価,初値予想_+200%以上,初値予想_+100%以上+200%未満,初値予想_+50%以上+100%未満,初値予想_+20%以上+50%未満,初値予想_+0%以上+20%未満,初値予想_+0%未満,会社設立,会社名,想定価格,仮条件,公募価格,初値予想（独自）,初値
0,3170,◎,〇,〇,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,アイセイ薬局（3170） 【JASDAQスタンダード】,"3,460円","3,300円 ～ 3,460円","3,460円",-,"3,460円（公募価格比 ±0％）"
1,3169,〇,△,〇,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ミサワ（3169） 【JASDAQスタンダード】,860円,760円 ～ 860円,860円,-,"1,209円（公募価格比 +40.6％）"
2,9206,〇,◎,〇,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,スターフライヤー（9206） 【東証2部】,"3,180円","2,900円 ～ 3,180円","3,180円",-,"5,500円（公募価格比 +73.0％）"
3,4977,△,〇,△,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,新田ゼラチン（4977） 【東証2部】,570円,500円 ～ 570円,500円,-,462円（公募価格比 -7.6％）
4,4583,△,△,△,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,カイオム・バイオサイエンス（4583） 【東証マザーズ】,"2,700円","2,100円 ～ 2,700円","2,700円",-,"2,480円（公募価格比 -8.1％）"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,4268,〇,△,〇,D,92.0,46.0,187.0,102.0,53.0,244.0,2014年（8年目）,エッジテクノロジー（4268） 【東証マザーズ】,690円,300円 ～ 350円,350円,322円 ～ 550円（2月8日時点）,694円 （公募価格比+344円 +98.3％）
1010,4267,◎,△,〇,C,116.0,28.0,53.0,59.0,103.0,224.0,1995年（27年目）,ライトワークス（4267） 【東証マザーズ】,"2,100円","1,900円 ～ 2,100円","2,100円","2,100円 ～ 2,900円（1月19日時点）","3,000円 （公募価格比+900円 +42.9％）"
1011,4266,〇,△,〇,C,69.0,16.0,15.0,28.0,79.0,255.0,NaN,ビッグツリーテクノロジー＆コンサルティング（4266） 【東証マザーズ】,"5,000円",未発表,未発表,"5,000円 ～ 6,000円（12月28日時点）",-
1012,9213,△,△,〇,D,53.0,16.0,19.0,40.0,80.0,192.0,1991年（31年目）,セイファート（9213） 【JASDAQスタンダード】,"1,120円","1,060円 ～ 1,120円","1,120円","1,030円 ～ 1,500円（1月26日時点）","1,030円 （公募価格比-90円 -8.0％）"


In [50]:
# IPOスケジュール + IPO当選株数 [3,4]　名称df_schedule
df_schedule = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
]],
df_scraping_table_all.loc[:, [
    "抽選申込期間",
    "当選発表日",
    "購入申込期間",
    "上場日",
    "公募株数",
    "売出株数（OA含む）",
    "当選株数合計"
]]],axis=1)
df_schedule

,code,抽選申込期間,当選発表日,購入申込期間,上場日,公募株数,売出株数（OA含む）,当選株数合計
0,3170,12月6日（火）～12月12日（月）,12月13日（火）,12月15日（木）～12月20日（火）,12月22日（木）,"249,000株","37,300株","286,300株"
1,3169,12月7日（水）～12月13日（火）,12月14日（水）,12月15日（木）～12月20日（火）,12月22日（木）,"380,000株","310,000株","690,000株"
2,9206,12月5日（月）～12月9日（金）,12月12日（月）,12月14日（水）～12月19日（月）,12月21日（水）,"140,000株","90,000株","230,000株"
3,4977,12月5日（月）～12月9日（金）,12月12日（月）,12月13日（火）～12月15日（木）,12月20日（火）,"3,000,000株","1,170,000株","4,170,000株"
4,4583,12月2日（金）～12月8日（木）,12月9日（金）,12月13日（火）～12月16日（金）,12月20日（火）,"188,000株","705,500株","893,500株"
...,...,...,...,...,...,...,...,...
1009,4268,2月1日(火)～2月7日(月),2月8日(火),2月9日(水)～2月15日(火),2月17日(木),"544,000株","2,862,900株","3,406,900株"
1010,4267,1月21日(金)～1月27日(木),1月28日(金),1月31日(月)～2月3日(木),2月9日(水),"160,000株","208,000株","368,000株"
1011,4266,1月20日(木)～1月26日(水),1月27日(木),1月28日(金)～2月2日(水),2月4日(金),"60,000株","4,213,400株","4,273,400株"
1012,9213,1月19日(水)～1月25日(火),1月26日(水),1月27日(木)～2月1日(火),2月4日(金),"420,800株","436,600株","857,400株"


In [52]:
df_scraping_table_all.columns.

Index(['会社名', '会社URL', '狙い目証券会社', '想定価格', '仮条件', '公募価格', '初値予想（独自）', '初値',
       '抽選申込期間', '当選発表日',
       ...
       '税引前利益（千米ドル）_5年目', '当期利益（千米ドル）_5年目', '純資産額 （千米ドル）_5年目',
       '1株あたりの純資産額（米ドル）_5年目', '1株あたりの純利益（米ドル）_5年目', '1株あたりの当期利益（米ドル）_1年目',
       '1株あたりの当期利益（米ドル）_2年目', '1株あたりの当期利益（米ドル）_3年目', '1株あたりの当期利益（米ドル）_4年目',
       '1株あたりの当期利益（米ドル）_5年目'],
      dtype='object', length=873)

In [65]:
df_scraping_table_all.filter(regex="幹事種類_|証券会社名_|割当率_|割当株数_|当選本数 （枚）_|完全抽選本数 （予想）_")

,幹事種類_1,証券会社名_1,割当率_1,割当株数_1,当選本数 （枚）_1,完全抽選本数 （予想）_1,幹事種類_2,証券会社名_2,割当率_2,割当株数_2,...,割当率_102,割当株数_102,当選本数 （枚）_102,完全抽選本数 （予想）_102,幹事種類_103,証券会社名_103,割当率_103,割当株数_103,当選本数 （枚）_103,完全抽選本数 （予想）_103
0,主幹事,大和証券,75.22％,"187,300株","1,873枚",187枚,幹事,SMBC日興証券,14.98％,"37,300株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,主幹事,みずほ インベスターズ 証券,80.00％,"480,000株","4,800枚",480枚,幹事,SMBC日興証券,10.00％,"60,000株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,主幹事,大和証券,75.00％,"150,000株","1,500枚",225枚,幹事,SMBC日興証券,5.00％,"10,000株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,主幹事,SMBC日興証券,74.99％,"2,720,700株","27,207枚","2,720枚",幹事,野村證券,9.00％,"326,600株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,主幹事,大和証券,80.04％,"621,900株","6,219枚",932枚,幹事,みずほ証券,4.00％,"31,100株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,主幹事,SMBC日興証券,93.05％,"2,899,900株","28,999枚","2,899枚",幹事,野村證券,4.35％,"135,500株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,主幹事,エイチ・エス証券,84.35％,"310,400株","3,104枚",310枚,幹事,SMBC日興証券,5.22％,"19,200株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,共同主幹事,三菱UFJ モルガン・ スタンレー証券,-％,-株,-枚,-枚,共同主幹事,大和証券,-％,-株,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,主幹事,みずほ証券,95.67％,"820,300株","8,203枚",820枚,幹事,大和証券,1.74％,"14,900株",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# 幹事証券リスト（管理人独自予想あり）[5]
df_underwriters = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
]],
df_scraping_table_all.filter(regex="幹事種類_|証券会社名_|割当率_|割当株数_|当選本数 （枚）_|完全抽選本数 （予想）_")
],axis=1)
df_underwriters

,code,幹事種類_1,証券会社名_1,割当率_1,割当株数_1,当選本数 （枚）_1,完全抽選本数 （予想）_1,幹事種類_2,証券会社名_2,割当率_2,...,割当率_102,割当株数_102,当選本数 （枚）_102,完全抽選本数 （予想）_102,幹事種類_103,証券会社名_103,割当率_103,割当株数_103,当選本数 （枚）_103,完全抽選本数 （予想）_103
0,3170,主幹事,大和証券,75.22％,"187,300株","1,873枚",187枚,幹事,SMBC日興証券,14.98％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3169,主幹事,みずほ インベスターズ 証券,80.00％,"480,000株","4,800枚",480枚,幹事,SMBC日興証券,10.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9206,主幹事,大和証券,75.00％,"150,000株","1,500枚",225枚,幹事,SMBC日興証券,5.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4977,主幹事,SMBC日興証券,74.99％,"2,720,700株","27,207枚","2,720枚",幹事,野村證券,9.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4583,主幹事,大和証券,80.04％,"621,900株","6,219枚",932枚,幹事,みずほ証券,4.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,4268,主幹事,SMBC日興証券,93.05％,"2,899,900株","28,999枚","2,899枚",幹事,野村證券,4.35％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,4267,主幹事,エイチ・エス証券,84.35％,"310,400株","3,104枚",310枚,幹事,SMBC日興証券,5.22％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,4266,共同主幹事,三菱UFJ モルガン・ スタンレー証券,-％,-株,-枚,-枚,共同主幹事,大和証券,-％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,9213,主幹事,みずほ証券,95.67％,"820,300株","8,203枚",820枚,幹事,大和証券,1.74％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# 株主構成 + ロックアップ[6]
df_shareholders = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
]],
df_scraping_table_all.filter(regex="株主名_|比率_|ロック アップ_|ロック  アップ_")
],axis=1)
df_shareholders

,code,株主名_1,比率_1,株主名_2,比率_2,株主名_3,比率_3,株主名_4,比率_4,株主名_5,...,ロック アップ_1,ロック アップ_2,ロック アップ_3,ロック アップ_4,ロック アップ_5,ロック アップ_6,ロック アップ_7,ロック アップ_8,ロック アップ_9,ロック アップ_10
0,3170,岡村 幸彦（社長）,30.41％,（株）望未,18.27％,メディパルホールディングス,18.27％,中央エム・リース,9.14％,SMBCキャピタル８号,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3169,三沢 太（社長）,76.15％,飯塚 智香,7.96％,尾張 睦,6.36％,大塚 幸江,4.09％,従業員持ち株会,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9206,"DCM IV, L.P.",17.26％,ＴＯＴＯ,5.08％,安川電機,3.29％,北九州エアターミナル,2.90％,九州電力,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4977,アイビーピー,25.10％,ニッタ,6.60％,三菱東京ＵＦＪ銀行,4.88％,三井住友銀行,4.71％,（有）以和貴,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4583,ファストトラックイニシアティブ１号,18.89％,藤原 正明（社長）,10.17％,オリンパスビジネスクリエイツ,8.59％,太田 邦史,5.65％,清田 圭一,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,4268,住本 幸士（社長）,93.56％,島田 雄太,3.09％,治田 知明,1.20％,坂西 茂,0.39％,生澤 剛士,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,4267,（株）エプシモーヴェ,53.21％,江口 夏郎（社長）,15.87％,小迫 宏行,7.33％,ライトワークス従業員持株会,4.62％,松林 洋太,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,4266,インテグラル３号投資事業有限責任組合,58.55％,大木 塁,11.23％,Innovation Alpha L.P.,9.29％,杉山 健（社長）,4.01％,インテグラル（株）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,9213,（株）ビューティープロスペリティー,32.28％,長谷川 高志（社長）,23.24％,長谷川 美栄,9.59％,山田 実,6.91％,藤本 宏志,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ここ以下で作成　12/13～

In [15]:
#初値予想アンケートの取得実験用　12/13～　これをもとに追加、その後削除する
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
url =  rf"https://www.ipokiso.com/company/2022/beex.html"
driver.get(url)
html_res = driver.page_source.encode('utf-8')
soup = bs4(html_res, 'html.parser')

C:\Users\xxp2p\AppData\Local\Temp/ipykernel_8728/132835294.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
C:\Users\xxp2p\AppData\Local\Temp/ipykernel_8728/132835294.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)


In [16]:
# アンケート価格リスト
soup.find_all("p",class_="vote-left")

[<p class="vote-left">5,070円以上</p>,
 <p class="vote-left">3,380円～5,070円</p>,
 <p class="vote-left">2,535円～3,380円</p>,
 <p class="vote-left">2,028円～2,535円</p>,
 <p class="vote-left">1,690円～2,028円</p>,
 <p class="vote-left">1,690円未満</p>]

In [17]:
# アンケートの票リスト
soup.find_all("div",class_="vote-bar")

[<div class="vote-bar" data-value="390"></div>,
 <div class="vote-bar" data-value="100"></div>,
 <div class="vote-bar" data-value="61"></div>,
 <div class="vote-bar" data-value="32"></div>,
 <div class="vote-bar" data-value="26"></div>,
 <div class="vote-bar" data-value="49"></div>]

In [18]:
len(soup.find_all("div",class_="vote-bar"))

6

In [19]:
vote_values_list = soup.find_all("p",class_="vote-left")
number_of_votes_list = soup.find_all("div",class_="vote-bar")

In [20]:
votes_list = []
for value in number_of_votes_list:
    # print(value)
    value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
    votes_list.append(value)

In [21]:
# driver.quit()

In [28]:
df_scraping_table_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_table_all.csv", encoding='utf-8_sig',index=False)

df_scraping_table_all

,会社名,会社URL,狙い目証券会社,想定価格,仮条件,公募価格,初値予想（独自）,初値,抽選申込期間,当選発表日,...,税引前利益（千米ドル）_5年目,当期利益（千米ドル）_5年目,純資産額 （千米ドル）_5年目,1株あたりの純資産額（米ドル）_5年目,1株あたりの純利益（米ドル）_5年目,1株あたりの当期利益（米ドル）_1年目,1株あたりの当期利益（米ドル）_2年目,1株あたりの当期利益（米ドル）_3年目,1株あたりの当期利益（米ドル）_4年目,1株あたりの当期利益（米ドル）_5年目
0,アイセイ薬局（3170） 【JASDAQスタンダード】,http://www.aisei.co.jp/,SMBC日興証券（副幹事）、SBI証券,"3,460円","3,300円 ～ 3,460円","3,460円",-,"3,460円（公募価格比 ±0％）",12月6日（火）～12月12日（月）,12月13日（火）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ミサワ（3169） 【JASDAQスタンダード】,http://www.unico-fan.co.jp/,SMBC日興証券（副幹事）、SBI証券、松井証券,860円,760円 ～ 860円,860円,-,"1,209円（公募価格比 +40.6％）",12月7日（水）～12月13日（火）,12月14日（水）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,スターフライヤー（9206） 【東証2部】,http://www.starflyer.jp/,SMBC日興証券（副幹事）、マネックス証券 カブドットコム証券,"3,180円","2,900円 ～ 3,180円","3,180円",-,"5,500円（公募価格比 +73.0％）",12月5日（月）～12月9日（金）,12月12日（月）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,新田ゼラチン（4977） 【東証2部】,http://www.nitta-gelatin.co.jp/,SMBC日興証券（主幹事）、SBI証券,570円,500円 ～ 570円,500円,-,462円（公募価格比 -7.6％）,12月5日（月）～12月9日（金）,12月12日（月）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,カイオム・バイオサイエンス（4583） 【東証マザーズ】,http://www.chiome.co.jp/,SBI証券、むさし証券、カブドットコム証券,"2,700円","2,100円 ～ 2,700円","2,700円",-,"2,480円（公募価格比 -8.1％）",12月2日（金）～12月8日（木）,12月9日（金）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,エッジテクノロジー（4268） 【東証マザーズ】,https://www.edge-tech.co.jp/,SMBC日興証券（主幹事）、野村證券（副幹事）、SBI証券、楽天証券、松井証券、DMM株,690円,300円 ～ 350円,350円,322円 ～ 550円（2月8日時点）,694円 （公募価格比+344円 +98.3％）,2月1日(火)～2月7日(月),2月8日(火),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,ライトワークス（4267） 【東証マザーズ】,https://www.lightworks.co.jp/,SMBC日興証券（副幹事）、SBI証券、楽天証券、マネックス証券、岡三オンライン、DMM株,"2,100円","1,900円 ～ 2,100円","2,100円","2,100円 ～ 2,900円（1月19日時点）","3,000円 （公募価格比+900円 +42.9％）",1月21日(金)～1月27日(木),1月28日(金),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,ビッグツリーテクノロジー＆コンサルティング（4266） 【東証マザーズ】,https://www.bigtreetc.com/,大和証券（共同主幹事）、SMBC日興証券（副幹事）、野村證券、SBI証券、マネックス証券、岩...,"5,000円",未発表,未発表,"5,000円 ～ 6,000円（12月28日時点）",-,1月20日(木)～1月26日(水),1月27日(木),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,セイファート（9213） 【JASDAQスタンダード】,https://www.seyfert.co.jp/,大和証券（副幹事）、岩井コスモ証券、マネックス証券、CONNECT、DMM株,"1,120円","1,060円 ～ 1,120円","1,120円","1,030円 ～ 1,500円（1月26日時点）","1,030円 （公募価格比-90円 -8.0％）",1月19日(水)～1月25日(火),1月26日(水),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


これを元に作成2022/12/09~ ↓　全銘柄チェック、その後、例外の銘柄のtableの変更実施


以上本番用

In [23]:
""" 
# csv変換 時短のためこれを読み込む
df_code = pd.DataFrame(code_list,columns=["code"])
df_scraping_url[df_code.columns] = df_code.values
df_scraping_evaluation.to_csv("df_scraping_evaluation", encoding='utf-8_sig')
"""

' \n# csv変換 時短のためこれを読み込む\ndf_code = pd.DataFrame(code_list,columns=["code"])\ndf_scraping_url[df_code.columns] = df_code.values\ndf_scraping_evaluation.to_csv("df_scraping_evaluation", encoding=\'utf-8_sig\')\n'

In [24]:
# 時短のためcode含めたcsv読み込み
df_scraping_evaluation = pd.read_csv("df_scraping_url.csv", index_col=0)
df_scraping_evaluation

FileNotFoundError: [Errno 2] No such file or directory: 'df_scraping_url.csv'

- IPO当選株数のデータ取得

backup

In [ ]:
# backup用
# requests.getでのデータ取得用ループ　評価
def make_company_info(url):
    html_res = requests.get(url)
    if html_res.status_code != 200:
        print("requests.getでのurlのアクセスができていません")
        lineno = inspect.currentframe().f_lineno
        print(f"エラーが発生しました。行番号: {lineno}")
        sys.exit()
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    val_list = []
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    val_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)
    print(code)
df_dict
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["code","成長性","割安性","話題性","総合評価"])
df_dict[df_company_01.columns] = df_company_01
print(df_dict)